Sublink processing hasn't been added (but is necessary)
selenium parsing may be useful but after some tests, it turns out to be not very useful

In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import warnings
from urllib.parse import urljoin
from urllib3.exceptions import InsecureRequestWarning

user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
]
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Define headers for requests
headers = {'User-Agent': random.choice(user_agents)}
session = requests.Session()

# Fetch the page text with retry mechanism
def fetch_page_text(url, retries=3, timeout=5):
    attempt = 0
    while attempt < retries:
        try:
            # Send a GET request to the URL with a timeout
            response = session.get(url, timeout=timeout, headers=headers, verify=False)
            response.raise_for_status()  # Raise an error for bad responses
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'lxml')
            
            # Extract and return the text from the page and the soup object
            return soup.get_text(separator='\n', strip=True), soup
        
        except requests.exceptions.RequestException as e:
            attempt += 1
            time.sleep(1)  # Optional: wait before retrying

        if attempt == retries:
            return None, None

# Extract all sublinks (anchor tags) and handle relative URLs
def extract_sublinks(soup, base_url):
    sublinks = []
    for link in soup.find_all('a', href=True):
        sublink = link['href']
        # Convert relative URLs to absolute URLs using urljoin
        full_url = urljoin(base_url, sublink)
        sublinks.append(full_url)
    return sublinks

# Crawl through the URLs and fetch the data with retry
def crawl_urls(url_list):
    results = {}
    sublinks_data = []  # List to store (parent_url, sublink) pairs
    for index, url in enumerate(url_list):
        print(f"Fetching: {url}")
        text, soup = fetch_page_text(url)
        if text and soup:
            # Store the crawled text
            results[url] = text
            # Extract sublinks
            sublinks = extract_sublinks(soup, url)
            for sublink in sublinks:
                sublinks_data.append((url, sublink))
        else:
            print(f"Failed to parse URL at index: {index}, URL: {url}")
    return results, sublinks_data

# Save the crawled text data to .txt files
def save_crawled_data(crawled_data, urls):
    for index, url in enumerate(urls):
        # Fetch the text corresponding to the URL from the crawled_data dictionary
        text = crawled_data.get(url, "")

        # Remove all newline characters from the text
        cleaned_text = text.replace('\n', ' ')

        # Define the file name with the index from the URLs list
        output_file = f"/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_text_data/{index}.txt"

        # Save the cleaned text to the file
        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(cleaned_text)

# Save sublinks to a CSV file
def save_sublinks_to_csv(sublinks_data, output_csv):
    df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
    df.to_csv(output_csv, index=False)

if __name__ == "__main__":
    file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
    data = pd.read_csv(file_path)

    # Extract non-empty URLs from the 'Source URL' column
    urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

    # Start crawling the URLs
    crawled_data, sublinks_data = crawl_urls(urls)
    
    # Save the crawled text data
    save_crawled_data(crawled_data, urls)

    # Save sublinks to a CSV file
    sublinks_output_file = "/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublinks.csv"
    save_sublinks_to_csv(sublinks_data, sublinks_output_file)

    print("Crawling complete!")

Fetching: https://en.wikipedia.org/wiki/Pittsburgh
Fetching: https://en.wikipedia.org/wiki/History_of_Pittsburgh
Fetching: https://pittsburghpa.gov/pittsburgh/pgh-about
Fetching: https://www.britannica.com/place/Pittsburgh
Fetching: https://pittsburghpa.gov/events/index.html
Fetching: https://www.visitpittsburgh.com/blog/move-in-day-pittsburgh-college-guide/
Fetching: https://pittsburghpa.gov/mayor/pghmayors
Fetching: https://en.wikipedia.org/wiki/Economy_of_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/Greater_Pittsburgh#
Fetching: https://en.wikipedia.org/wiki/Government_of_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/Transportation_in_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/List_of_colleges_and_universities_in_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/Timeline_of_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/List_of_people_from_Pittsburgh
Fetching: https://en.wikipedia.org/wiki/List_of_corporations_in_Pittsburgh
Fetching: https://pittsburghpa.gov/

The below cell create a mapping between the parent link file names (in index numbers) and the actual urls.

In [18]:
file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/raw/raw_csv_data/data_source.csv'
data = pd.read_csv(file_path)

# Extract non-empty URLs from the 'Source URL' column
urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

import csv

output_csv = '/Users/alan/11711/nlp-from-scratch-assignment/data/crawled/parentlink_file_name_url_mapping.csv'

# Create the CSV mapping file
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header (optional)
    writer.writerow(['Index', 'Value'])
    
    # Write the index and value pairs to the CSV
    for index, value in enumerate(urls):
        writer.writerow([index, value])

print(f"CSV mapping file created at {output_csv}")

CSV mapping file created at /Users/alan/11711/nlp-from-scratch-assignment/data/crawled/parentlink_file_name_url_mapping.csv


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import warnings
from urllib.parse import urljoin
from urllib3.exceptions import InsecureRequestWarning
from tqdm import tqdm

# Define user agents
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
]
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Define headers for requests
headers = {'User-Agent': random.choice(user_agents)}
session = requests.Session()

# Fetch the page text with retry mechanism
def fetch_page_text(url, retries=1, timeout=5):
    attempt = 0
    while attempt < retries:
        try:
            # Send a GET request to the URL with a timeout
            response = session.get(url, timeout=timeout, headers=headers, verify=False)
            response.raise_for_status()  # Raise an error for bad responses
            
            # Parse the page content
            soup = BeautifulSoup(response.content, 'lxml')
            
            # Extract and return the text from the page and the soup object
            return soup.get_text(separator='\n', strip=True), soup
        
        except requests.exceptions.RequestException as e:
            attempt += 1

        if attempt == retries:
            return None, None

# Save the crawled text data to .txt files
def save_crawled_text(url, text, index):
    # Clean up the text by removing newlines
    cleaned_text = text.replace('\n', ' ')

    # Define the file name with the index from the URLs list
    output_file = f"/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublink_text_data/{index}.txt"

    # Save the cleaned text to the file
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

# Crawl through the URLs, parse and save one by one
def crawl_and_save(url_list):
    for index, url in tqdm(enumerate(url_list), total=len(url_list)):
        if url.startswith('http'):
            text, _ = fetch_page_text(url)
            if text:
                # Save the parsed text immediately after parsing
                save_crawled_text(url, text, index)
            else:
                print(f"Failed to parse URL at index {index}, URL: {url}")

if __name__ == "__main__":
    # Read the CSV file with URLs
    file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublinks.csv'
    data = pd.read_csv(file_path)

    # Extract non-empty URLs from the 'Source URL' column
    urls = data['Sublink'].dropna().unique()

    # Start crawling the URLs and save the result one by one
    crawl_and_save(urls)

    print("Crawling complete!")

  9%|▉         | 1629/18004 [12:02<2:23:07,  1.91it/s]

Failed to parse URL at index 1628, URL: https://en.wikipedia.org/w/index.php?title=Pennsylvania_American_Water&action=edit&redlink=1


 10%|█         | 1885/18004 [14:05<9:59:34,  2.23s/it]

Failed to parse URL at index 1884, URL: https://web.archive.org/web/20230811122527/https://triblive.com/local/regional/lawrence-county-added-to-pittsburgh-metro-area/


 10%|█         | 1890/18004 [14:07<3:18:24,  1.35it/s]

...

Failed to parse URL at index 17996, URL: https://www.wtrf.com/pennsylvania/watch-man-breaks-record-for-chugging-pickle-juice-in-pennsylvania/


100%|█████████▉| 17998/18004 [3:47:59<00:14,  2.45s/it]

Failed to parse URL at index 17997, URL: https://web.archive.org/web/20231101034853/https://www.wtrf.com/pennsylvania/watch-man-breaks-record-for-chugging-pickle-juice-in-pennsylvania/


100%|██████████| 18004/18004 [3:48:02<00:00,  1.32it/s]

Crawling complete!


The below cell create a mapping between the sublink file names (in index numbers) and the actual urls.

In [16]:
import csv

output_csv = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/sublink_file_name_url_mapping.csv'

# Create the CSV mapping file
with open(output_csv, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header (optional)
    writer.writerow(['Index', 'Value'])
    
    # Write the index and value pairs to the CSV
    for index, value in enumerate(urls):
        writer.writerow([index, value])

print(f"CSV mapping file created at {output_csv}")

CSV mapping file created at /Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/sublink_file_name_url_mapping.csv


In [ ]:
import pandas as pd
from urllib.parse import urlparse, urlunparse

# Function to normalize a URL by removing the fragment (i.e., anything after #)
def normalize_url(url):
    parsed_url = urlparse(url)
    # Reconstruct the URL without the fragment
    return urlunparse(parsed_url._replace(fragment=''))

# Function to check and remove duplicate URLs based on the base URL (without fragments)
def remove_duplicate_urls(file_path, column_name):
    # Read the CSV file
    data = pd.read_csv(file_path)

    # Normalize the URLs (i.e., remove the fragment part)
    data['normalized_url'] = data[column_name].apply(normalize_url)

    # Remove duplicate rows based on the 'normalized_url' column
    data_cleaned = data.drop_duplicates(subset='normalized_url', keep='first')

    # Drop the 'normalized_url' column as it's no longer needed
    data_cleaned = data_cleaned.drop(columns=['normalized_url'])

    # Save the cleaned data back to a CSV file
    output_file = file_path.replace('.csv', '_cleaned.csv')
    data_cleaned.to_csv(output_file, index=False)

    print(f"Duplicates removed. Cleaned data saved to {output_file}")

if __name__ == "__main__":
    # Path to the CSV file
    file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_sublinks.csv'
    
    # Column in the CSV file containing the URLs
    column_name = 'Source URL'  # Adjust based on your CSV column name

    # Remove duplicates
    remove_duplicate_urls(file_path, column_name)

Below is the code for Selenium web driver crawling (which is evetually not used).

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

# Function to fetch the page content using Selenium
def fetch_page_text_selenium(url):
    try:
        driver = webdriver.Chrome(driverPath) 
        # Initialize Chrome WebDriver
        driver.get(url)
        
        # Wait for the page to load (adjust time if needed)
        time.sleep(3)
        
        # Get page source and close the browser
        page_source = driver.page_source
        driver.quit()

        # Parse the page content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Extract and clean the text from the page
        page_text = soup.get_text(separator='\n', strip=True)
        return page_text

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

# Function to read URLs from CSV and crawl each one
def crawl_urls_from_csv(csv_file_path, url_column_name):
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for index, row in enumerate(reader):
            url = row[url_column_name]
            print(f"Fetching: {url}")
            text = fetch_page_text_selenium(url)
            if text:
                # Save the crawled text to a file with the index as the filename
                output_file = f"/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/crawled/crawled_text_data/{index}.txt"
                with open(output_file, 'w', encoding='utf-8') as f:
                    f.write(text)
                print(f"Saved content to {output_file}")

if __name__ == "__main__":
    csv_file_path = '/Users/alan/11711/nlp-from-scratch-assignment/data/1010_160_entries/raw/raw_csv_data/data_source.csv'
    url_column_name = 'Source URL'
    driverPath = '/Users/alan/Downloads/chromedriver-mac-arm64/chromedriver'


    # Start crawling the URLs from the CSV
    crawl_urls_from_csv(csv_file_path, url_column_name)